# KE5207 CA1 Genetic Algorithm Modelling - Data Modelling

## Load libraries

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from deap import base, creator, tools
from deap.algorithms import eaSimple
import myUtilities as mu
import random
import datetime
import pickle
import multiprocessing
import functools

## Load data

In [3]:
gifts_df = pd.read_csv(os.path.join('data', 'gifts.csv'), header=0)
gifts_df.head()

,GiftId,Latitude,Longitude,Weight
0,1,16.345769,6.303545,1.000000
1,2,12.494749,28.626396,15.524480
2,3,27.794615,60.032495,8.058499
3,4,44.426992,110.114216,1.000000
4,5,-69.854088,87.946878,25.088892


In [4]:
gifts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 4 columns):
GiftId       100000 non-null int64
Latitude     100000 non-null float64
Longitude    100000 non-null float64
Weight       100000 non-null float64
dtypes: float64(3), int64(1)
memory usage: 3.1 MB


In [5]:
# Split a train dataset.
X_train, X_test = train_test_split(gifts_df.values, test_size=.99, shuffle=True, random_state=42)
X_train[:5]

array([[  7.60910000e+04,   1.09924090e+01,   7.71507936e+01,
          1.00000000e+00],
       [  7.57040000e+04,  -1.63775119e+01,   4.57377035e+01,
          1.00000000e+00],
       [  3.45300000e+04,   5.47971627e+01,  -9.62457396e+01,
          1.00000000e+00],
       [  4.84480000e+04,  -7.41267480e+01,   4.73413759e+01,
          1.36050152e+01],
       [  5.45540000e+04,  -7.29671349e+00,  -6.22714911e+01,
          1.11474524e+01]])

In [6]:
len(X_train)

1000

## Optimisation Using Genetic Algorithm

### Individuals and population

In [7]:
# Create the types for the chromosomes and population.
creator.create('FitnessMin', base.Fitness, weights=(-1.0,))
creator.create('Individual', list, fitness=creator.FitnessMin)

In [8]:
# Initialise the toolbox.
toolbox = base.Toolbox()

In [9]:
# Create the function for initialising an individual.
toolbox.register('create_indiv', mu.create_indiv, gifts=X_train[:,0].tolist())
# toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.create_indiv, n=1)
toolbox.register("individual", mu.create_indiv, icls=creator.Individual, gifts=X_train[:, 0].tolist())

In [10]:
# Create the function for initialising a population.
toolbox.register("population", tools.initRepeat, list, toolbox.individual, n=50)

### Fitness function

In [11]:
# Evaluation or fitness function
def evaluate(indiv):
    return (mu.weighted_reindeer_weariness(indiv),)

### Mutation functions

In [12]:
# Mutation function 07

# All gifts from a randomly selected trip are reinserted one by one in the order used for picking the initial route (i.e. by sequential insertion).
# def mutate_07(indiv):
#     mutant = []
#     return mutant

In [13]:
# Mutation function 08

# A randomly selected trip is divided into 2 trips using 1 gift as the cut point.
def mutate_08(indiv):

    mutant = indiv.copy()  # if mutation fails, return the same individual, here mutant is converted to type list
    mutant_idx = list(range(0, len(mutant)))
    not_mutated = True

    while (mutant_idx and not_mutated):

        selected_trip_idx = random.sample(mutant_idx, 1)[0]
        selected_trip = mutant[selected_trip_idx]

        if len(selected_trip) > 1:  # perform mutation

            selected_gift_idx = random.randint(0, len(selected_trip) - 1)

            if selected_gift_idx > 0:
                mutated_trip = [selected_trip[:selected_gift_idx], selected_trip[selected_gift_idx:]]
            else:  # special case of index 0
                mutated_trip = [[selected_trip[0]], selected_trip[1:]]

            if selected_trip_idx == 0:  # first trip
                mutant = mutated_trip + mutant[1:]
            elif selected_trip_idx == (len(mutant)-1):  # last trip
                mutant = mutant[:-1] + mutated_trip
            else:  # trip in middle of indiv
                mutant = mutant[:selected_trip_idx] + mutated_trip + mutant[selected_trip_idx + 1:]

            not_mutated = False  # mutation completed

        else:  # cannot mutate because only 1 gift in trip, look for another trip
            mutant_idx.remove(selected_trip_idx)

    ##### for testing
    # for i in mutant:
    #     if not i:
    #         print('empty')
    return ((type(indiv))(mutant),)

Check the mutation function 08

In [14]:
t1 = datetime.datetime.now()
indiv = mu.create_indiv(creator.Individual, X_train[:,0].tolist())
t2 = datetime.datetime.now()
print(t2-t1)

KeyboardInterrupt: 

In [26]:
# Save individual to file for testing.
save_to_file = open(os.path.join('data', 'individual1.pickle'), 'wb')
pickle.dump(list(indiv), save_to_file)
save_to_file.close()

In [18]:
t1 = datetime.datetime.now()
mut = mutate_08(indiv)
t2 = datetime.datetime.now()
print(t2-t1)

0:00:00.000113


In [37]:
len(indiv)

174

In [38]:
len(mut[0])

175

In [41]:
for index, i in enumerate(zip(indiv, mut[0])):
    if i[0] != i[1]:
        print(index, i[0], i[1])
        break

153 [27533.0, 99361.0, 1016.0] [27533.0, 99361.0]


The 154th trip is mutated.

### Cross-over functions 

Cross-over function mate() defined in myUtilities.py


### Genetic Algorithm

In [15]:
# Operators
toolbox.register("mate", mu.mate)
toolbox.register("mutate", mutate_08)
toolbox.register("select", tools.selBest)
toolbox.register("evaluate", evaluate)

In [16]:
# Statistics
s = tools.Statistics()
s.register('avg', np.mean)
s.register("std", np.std)
s.register("min", np.min)
s.register("max", np.max)

In [24]:
# Create the initial population using the sequential insertion heuristic.
t1 = datetime.datetime.now()

# Use multiprocessing for creating initial population
population_size = 5
pool = multiprocessing.Pool()
init_pop = multiprocessing.Manager().list()
func = functools.partial(mu.mp_create_indiv, creator.Individual, X_train[:,0].tolist(), init_pop)

pool.map(func, range(population_size))
pool.close()  # close the pool
pool.join()  # wait for the pool to close
#init_pop = toolbox.population()

t2 = datetime.datetime.now()
print(t2-t1)

0:19:30.811394


In [27]:
len(init_pop)

5

In [28]:
total_gift_weight = 0
for i in init_pop[0][0]:
    total_gift_weight += mu.gift_weight[i]
total_gift_weight

89.83547152281999

In [29]:
total_gifts = 0
all_gifts = []
for i in init_pop[0]:
    total_gifts += len(i)
    all_gifts = all_gifts + i
total_gifts

1000

In [31]:
[len(i) for i in init_pop[0]][-10:]

[3, 2, 2, 2, 2, 2, 2, 1, 1, 1]

In [36]:
# Check the weight of the last 2 gifts.
print(init_pop[0][-2], mu.gift_weight[init_pop[0][-2][0]], init_pop[0][-1], mu.gift_weight[init_pop[0][-1][0]])

[88727.0] 50.0 [28700.0] 50.0


This is correct because there is a base weigh of 10 for the sleigh.

In [38]:
# Save individual to file for testing.
save_to_file = open(os.path.join('data', 'init_pop_n5_len1000.pickle'), 'wb')
pickle.dump(init_pop, save_to_file)
save_to_file.close()

In [22]:
# Main genetic algorithm
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

t1 = datetime.datetime.now()
pop, logbook = eaSimple(init_pop, toolbox, cxpb=.5, mutpb=.02, ngen=100, stats=s, verbose=True)
t2 = datetime.datetime.now()
print(t2-t1)

TypeError: unhashable type: 'list'

In [21]:
init_pop

functools.partial(<function initRepeat at 0x7fc04bb520d0>, <class 'list'>, functools.partial(<function initRepeat at 0x7fc04bb520d0>, <class 'deap.creator.Individual'>, functools.partial(<function create_indiv at 0x7fc04bb6d730>, gifts=[40775.0, 48985.0, 61229.0, 51215.0, 38045.0, 8572.0, 39100.0, 86780.0, 63705.0, 12186.0, 86808.0, 9269.0, 82799.0, 74066.0, 24301.0, 23248.0, 89813.0, 55592.0, 89790.0, 1017.0, 91388.0, 40398.0, 94664.0, 11535.0, 32607.0, 99300.0, 92094.0, 37066.0, 65698.0, 71212.0, 72410.0, 45759.0, 9693.0, 40758.0, 52996.0, 78954.0, 77190.0, 66558.0, 19458.0, 89476.0, 69480.0, 67122.0, 92068.0, 35921.0, 80078.0, 17160.0, 48556.0, 23484.0, 68149.0, 23898.0, 76553.0, 43002.0, 73970.0, 8793.0, 10628.0, 87314.0, 80039.0, 96277.0, 41607.0, 3891.0, 69093.0, 11395.0, 31552.0, 66804.0, 56887.0, 67436.0, 35774.0, 84655.0, 65726.0, 59151.0, 2748.0, 18432.0, 84479.0, 25659.0, 93017.0, 71933.0, 28694.0, 85306.0, 53708.0, 83105.0, 5312.0, 67970.0, 64926.0, 62956.0, 59736.0, 770.0,

In [46]:
total_gift_weight = 0
for i in indiv[0]:
    total_gift_weight += mu.gift_weight[i]
total_gift_weight

89.95614644289999

In [49]:
total_gifts = 0
all_gifts = []
for i in indiv:
    total_gifts += len(i)
    all_gifts = all_gifts + i
total_gifts

1000